<a href="https://colab.research.google.com/github/JGS2020-012/text_analysis/blob/master/calcNegaPosiScore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import csv
import requests
import io


def load_dict(url_file):
  res = requests.get(url_file).content
  lists = []
  reader = csv.reader(io.StringIO(res.decode('utf-8')), delimiter='\t')
  for row in reader:
    #print(reader.line_num,row[0])
    lists.append(row)
  return lists

In [0]:
# 極性表現辞書をSNSテキストと同条件の形態素解析の原形を取得
def normalize_dict(dict_list):
  preprocess

In [0]:
# ネガポジ表現数を基にスコアリング
def calc_score(input_text):
  preprocess()

In [0]:
# 形態素解析
def wakati(input_text):
  preprocess()

In [20]:
# テストプログラム
term_lists = load_dict('http://www.cl.ecei.tohoku.ac.jp/resources/sent_lex/wago.121808.pn')
noun_lists = load_dict('http://www.cl.ecei.tohoku.ac.jp/resources/sent_lex/pn.csv.m3.120408.trim')

['ネガ（経験）', 'あがく']
